In [22]:
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup

# Importing selenium webdriver 
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')

# Importing required Exceptions which needs to handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.common.by import By
#Importing requests
import requests

# importing regex
import re

# 1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search 
for guitars

In [23]:
driver = webdriver.Chrome(r"C:\Users\Yashika\OneDrive\Desktop\chromedriver.exe")

In [24]:
url = "https://www.amazon.in/"
driver.get(url)

time.sleep(2)
# input the keywords that user wants to search
user_input = input('Enter the product you want to search : ')

Enter the product you want to search : Guitar


In [25]:
#Scraping search bar Xpath and clicking on search icon
search_bar = driver.find_element(By.ID,"twotabsearchtextbox") 
# to clear the search_bar
search_bar.clear() 
# to send user input to search bar
search_bar.send_keys(user_input)
 # to Locate search_button by xpath
search_button = driver.find_element(By.XPATH,'//div[@class="nav-search-submit nav-sprite"]/span/input')      
search_button.click()  

# Ques 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“

In [26]:
#collecting all the Product URLS
urls = []
for i in range(0,3):
    Page_urls=driver.find_elements(By.XPATH,'//a[@class="a-link-normal s-no-outline"]')#collecting urls of all the guitar
    for i in Page_urls:
        urls.append(i.get_attribute('href'))
        
    #next button 
    nxt_btn=driver.find_element(By.XPATH,'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]') 
    url=nxt_btn.get_attribute('href')
    driver.get(url)
    time.sleep(2)

In [27]:
#Making empty lists and scraping the required spots
Product_name = []
Brand_name= []
Price = []
Return = []
Expected_Delivery = []
Availability = []

#Start with for loop
for i in urls:
    driver.get(i)
    time.sleep(2)
    
    
    #Scraping product name
    try:
        prod=driver.find_element(By.XPATH,'//span[@class="a-size-base-plus a-color-base a-text-normal"]')
        Product_name.append(prod.text)
    except NoSuchElementException as e:
        Product_name.append("-")
        
        
    #Scraping brand name
    try:
        brand=driver.find_element(By.XPATH,'//div[@id="bylineInfo_feature_div"]/div/a')
        Brand_name.append(brand.text)
    except NoSuchElementException as e:
        Brand_name.append("-")
        
        
    #Scraping Price
    try:
        pri=driver.find_element(By.XPATH,'//span[@class="a-price-whole"]')  
        Price.append(pri.text)
    except NoSuchElementException as e:
        Price.append("-")
        
        
    #Scraping Return/Exchange
    try:
        ret=driver.find_element(By.XPATH,'//div[@data-name="RETURNS_POLICY"]/span/div[2]/a')
        Return.append(ret.text)
    except NoSuchElementException as e:
        Return.append("-")
        
     
    #Scraping  Expected_Delivery
    try:
        delivary=driver.find_element(By.XPATH,'//span[@class="a-color-base a-text-bold"]')
        Expected_Delivery.append(delivary.text)
    except NoSuchElementException as e:
        Expected_Delivery.append("-")
        
        
    #Scraping Availability
    try:
        avai=driver.find_element(By.XPATH,'//div[@id="availability"]')
        Availability.append(avai.text)
    except NoSuchElementException as e:
        Availability.append("-")

        
# To make dataframe
Guitar=pd.DataFrame({})
Guitar['Name'] = Product_name
Guitar['Brand'] = Brand_name
Guitar['Price'] = Price
Guitar['Return'] = Return
Guitar['Expected_Delivery'] = Expected_Delivery
Guitar['Availability'] = Availability
Guitar['Urls'] = urls

In [28]:
Guitar

,Name,Brand,Price,Return,Expected_Delivery,Availability,Urls
0,Tygot Adjustable Aluminium Alloy Tripod Stand ...,-,797,-,"Tomorrow, January 29",-,https://www.amazon.in/Musical-Instruments/b/re...
1,-,Visit the VAULT Store,"4,240",7 days Replacement,-,In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,-,Brand: Hontral,899,7 days Replacement,-,In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,-,Brand: Generic,"9,990",7 days Returnable,-,Only 1 left in stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,-,Visit the VAULT Store,"6,749",7 days Replacement,-,In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
...,...,...,...,...,...,...,...
158,-,Visit the Tanglewood Store,"13,999",7 days Replacement,-,In stock.,https://www.amazon.in/Tanglewood-Crossroads-Ac...
159,-,Brand: Cort,"33,197",7 days Replacement,-,In stock.,https://www.amazon.in/Cort-CR250TBK-Classic-El...
160,-,Brand: Gibz's guitar,"6,250",7 days Replacement,-,Only 1 left in stock.,https://www.amazon.in/Gibzs-G-39-Glossy-Acoust...
161,-,Brand: Pluto,"6,900",7 days Replacement,-,In stock.,https://www.amazon.in/Pluto-HW41C201-HW41C-201...


In [29]:
driver.close

<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="6c200f5934033e899b89b0bb5158e773")>>

# Q3.Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [30]:
driver = webdriver.Chrome(r"C:\Users\Yashika\OneDrive\Desktop\chromedriver.exe")

In [31]:
# Connecting to the webdriver
driver=webdriver.Chrome(r"chromedriver.exe")
# Getting the webpage of mentioned url 
url = "https://images.google.com/"
#Creating empty list 
urls = []    #empty list
data = []    #empty list
search_item = ["fruits", "cars", "Machine Learning", "Guitar", "Cakes"]
for item in search_item:
    driver.get(url)  
    time.sleep(5)
    
    # Finding webelement for search_bar
    search_bar = driver.find_element(By.TAG_NAME,"input") 
    
    # Sending keys to get the keyword for search bar
    search_bar.send_keys(str(item))
    
    # Clicking on search button
    search_button = driver.find_element(By.XPATH,'//button[@class="Tg7LZd"]').click()
    
    # Scrolling down the webpage to get some more images
    for _ in range(50):
        driver.execute_script("window.scrollBy(0,100)")
        
        imgs = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')
    img_url = []
    for image in imgs:
        source = image.get_attribute('src')
        if source is not None:
                if(source[0:4] == 'http'):
                    img_url.append(source)
    for i in img_url[:10]:
        urls.append(i)
                    
for i in range(len(urls)):
    if i >= 50:
        break
    print("Downloading {0} of {1} images" .format(i, 50))
    response = requests.get(urls[i])
    file = open(r"C:\Users\yashika\fliprobo\google pics"+str(i)+".jpg","wb")
    file.write(response.content)

ElementClickInterceptedException: Message: element click intercepted: Element <button jsname="Tg7LZd" class="Tg7LZd" aria-label="Google Search" type="submit" data-ved="0ahUKEwj53dq39uz8AhUjTGwGHa9vD4cQ4dUDCAc">...</button> is not clickable at point (787, 214). Other element would receive the click: <iframe role="presentation" frameborder="0" scrolling="no" name="callout" src="https://ogs.google.com/widget/callout?prid=19028915&amp;pgid=19027681&amp;puid=3e9e72cbc9a83b93&amp;cce=1&amp;dc=1&amp;origin=https%3A%2F%2Fimages.google.com&amp;cn=callout&amp;pid=1&amp;spid=2&amp;hl=en" style="height: 100%; width: 100%;" data-iml="1675000319163"></iframe>
  (Session info: chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x00596643]
	(No symbol) [0x0052BE21]
	(No symbol) [0x0042DA9D]
	(No symbol) [0x004692B2]
	(No symbol) [0x00466970]
	(No symbol) [0x00463E0B]
	(No symbol) [0x00462548]
	(No symbol) [0x00456B13]
	(No symbol) [0x0047FD7C]
	(No symbol) [0x0045641F]
	(No symbol) [0x004800D

#  Question - 4 Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) onwww.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [32]:
driver = webdriver.Chrome(r"C:\Users\Yashika\OneDrive\Desktop\chromedriver.exe")

In [33]:
url = "https://www.flipkart.com/"
driver.get(url)
time.sleep(3)

In [34]:
driver.find_element(By.XPATH,"//button[@class='_2KpZ6l _2doB4z']").click()

In [35]:
item = input(" Enter the name of Smartphone that has to be searched : ")

#giving input key word to search bar
serch_bar = driver.find_element(By.XPATH,"//div[@class='_3OO5Xc']//input")
serch_bar.send_keys(item)

# to click search button
srch_btn = driver.find_element(By.XPATH,"/html/body/div[1]/div/div[1]/div[1]/div[2]/div[2]/form/div/button")
srch_btn.click()
time.sleep(5)

 Enter the name of Smartphone that has to be searched : ONEPLUS NORD


In [36]:
page1_urls = []
urls = driver.find_elements(By.XPATH,'//a[@class="_1fQZEK"]')
for url in urls:
    page1_urls.append(url.get_attribute("href"))

In [37]:
len(page1_urls)

24

In [38]:
# to make emoty list for all features to be scraped
Smartphones = {}
Smartphones["Brand"] = []
Smartphones["Phone name"] = []
Smartphones["Colour"] = []
Smartphones["RAM"] = []
Smartphones["Storage(ROM)"] = []
Smartphones["Primary Camera"] = []
Smartphones["Secondary Camera"] = []
Smartphones["Display Size"] = []
Smartphones["Battery Capacity"] = []
Smartphones["Price"] = []
Smartphones["URL"] = []

In [39]:
# Scraping data from each url of page 1
for url in page1_urls:
    driver.get(url)                                                        
    print("Scraping URL = ", url)
    Smartphones['URL'].append(url)                                                          
    time.sleep(2)
    
    
    #Clicking on read more button
    try:
        read_more = driver.find_element(By.XPATH,'//button[@class="_2KpZ6l _1FH0tX"]')     
        read_more.click()
    except NoSuchElementException:
        print("Exception occured while moving to next page")
    
    
    #Scraping brand name of phone data
    try:
        brand_tags = driver.find_element(By.XPATH,'//span[@class="B_NuCI"]')      
        Smartphones["Brand"].append(brand_tags.text.split()[0])
    except NoSuchElementException:
        Smartphones['Brand'].append('-')
    
    
    #Scraping phone name data
    try:
        name_tags = driver.find_element(By.XPATH,'//li[@class="_21lJbe"]')     
        Smartphones['Phone name'].append(name_tags.text)
    except NoSuchElementException:
        Smartphones['Phone name'].append('-')
    
    
    #Scraping phone color data
    try:
        color_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')      
        Smartphones['Colour'].append(color_tags.text)
    except NoSuchElementException:
        Smartphones['Colour'].append('-')
        
      #Scraping RAM data
    try:
        ram_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[2]/td[2]/ul/li')                
        Smartphones['RAM'].append(ram_tags.text)
    except NoSuchElementException:
        Smartphones['RAM'].append('-')
    
    
    #Scraping ROM data
    try:
        rom_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[1]/td[2]/ul/li')        
        Smartphones['Storage(ROM)'].append(rom_tags.text)
    except NoSuchElementException:
        Smartphones['Storage(ROM)'].append('-')
        
        
    #Scraping Primary camera data
    try:                                                                                    
        pri_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[2]/td[2]/ul/li')
        Smartphones['Primary Camera'].append(pri_tags.text)
    except NoSuchElementException:
        Smartphones['Primary Camera'].append('-')
        
        
    #Scraping secondary camera data
    try:                                                                                    
        sec_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[1]')
        if sec_tags != "Secondary Camera" : 
            if driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[1]').text == "Secondary Camera":
                sec_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[2]/ul/li')
            else :
                raise NoSuchElementException
        else :
            sec_cam = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[2]/ul/li')
        Smartphones['Secondary Camera'].append(sec_cam.text)
    except NoSuchElementException:
        Smartphones['Secondary Camera'].append('-')
        
    
    #Scraping Display size data 
    try:
        disp_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][2]/div')
        if disp_tags.text != "Display Features" : raise NoSuchElementException
        disp_size = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[1]/td[2]/ul/li')  
        Smartphones['Display Size'].append(disp_size.text)
    except NoSuchElementException:
        Smartphones['Display Size'].append('-')
     

    
    #Scraping battery capacity data
    try:
        if driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[2]/ul/li')                
            elif driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
            if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
            bat_capa = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')              
        Smartphones['Battery Capacity'].append(bat_capa.text)
    except NoSuchElementException:
        Smartphones['Battery Capacity'].append('-')
        
     #Scraping Price data
    try:
        price_tags = driver.find_element(By.XPATH,'//div[@class="_30jeq3 _16Jk6d"]')      
        Smartphones['Price'].append(price_tags.text)
    except NoSuchElementException:
        Smartphones['Price'].append('-')
        

Scraping URL =  https://www.flipkart.com/oneplus-nord-ce-2-lite-5g-blue-tide-128-gb/p/itm7acae55b999e6?pid=MOBGHBNGY5NDH3GV&lid=LSTMOBGHBNGY5NDH3GVAKFUJD&marketplace=FLIPKART&q=ONEPLUS+NORD&store=tyy%2F4io&spotlightTagId=TrendingId_tyy%2F4io&srno=s_1_1&otracker=search&otracker1=search&fm=Search&iid=24b208b8-2aec-4b6c-b648-3e5e9693aebc.MOBGHBNGY5NDH3GV.SEARCH&ppt=sp&ppn=sp&ssid=mo5zyzb74w0000001675000535162&qH=93fc5fde505957dd
Scraping URL =  https://www.flipkart.com/oneplus-nord-ce-2-lite-5g-blue-tide-128-gb/p/itm7acae55b999e6?pid=MOBGKGRADDHG3PN4&lid=LSTMOBGKGRADDHG3PN4EVYXS2&marketplace=FLIPKART&q=ONEPLUS+NORD&store=tyy%2F4io&srno=s_1_2&otracker=search&otracker1=search&fm=organic&iid=24b208b8-2aec-4b6c-b648-3e5e9693aebc.MOBGKGRADDHG3PN4.SEARCH&ppt=hp&ppn=homepage&ssid=mo5zyzb74w0000001675000535162&qH=93fc5fde505957dd
Scraping URL =  https://www.flipkart.com/oneplus-nord-ce-2-lite-5g-blue-tide-128-gb/p/itm7acae55b999e6?pid=MOBGJMUHHJE55ARU&lid=LSTMOBGJMUHHJE55ARUZBZB6I&marketplace=FLI

In [41]:
#Checking lengths of all scraped data
print(len(Smartphones["Brand"]), len(Smartphones["Phone name"]), len(Smartphones["Colour"]), len(Smartphones["RAM"]), len(Smartphones["Storage(ROM)"]))


24 24 24 24 24


In [42]:
#Framing the data
df = pd.DataFrame.from_dict(Smartphones)
df

,Brand,Phone name,Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Battery Capacity,Price,URL
0,OnePlus,"Phone,SIM Tray Ejector, Adapter, Phone Case, U...",Blue Tide,8 GB,128 GB,-,-,16.74 cm (6.59 inch),-,"₹21,995",https://www.flipkart.com/oneplus-nord-ce-2-lit...
1,OnePlus,"Phone,SIM Tray Ejector, Adapter, Phone Case, U...",Blue Tide,6 GB,128 GB,Primary Camera,-,16.74 cm (6.59 inch),-,"₹19,999",https://www.flipkart.com/oneplus-nord-ce-2-lit...
2,OnePlus,"Phone,SIM Tray Ejector, Adapter, Phone Case, U...",Blue Tide,6 GB,128 GB,Primary & Secondary Camera,-,16.74 cm (6.59 inch),-,"₹19,997",https://www.flipkart.com/oneplus-nord-ce-2-lit...
3,OnePlus,"Phone,SIM Tray Ejector, Adapter, Phone Case, U...",Blue Tide,6 GB,128 GB,-,-,16.74 cm (6.59 inch),-,"₹18,500",https://www.flipkart.com/oneplus-nord-ce-2-lit...
4,OnePlus,"Phone,SIM Tray Ejector, Adapter, Phone Case, U...",Black Dusk,6 GB,128 GB,-,-,16.74 cm (6.59 inch),-,"₹19,998",https://www.flipkart.com/oneplus-nord-ce-2-lit...
5,OnePlus,"Handset , 65W SUPERVOOC power adapter, Cable (...",Bahama Blue,8 GB,128 GB,Primary Camera,-,16.33 cm (6.43 inch),-,"₹24,998",https://www.flipkart.com/oneplus-nord-ce-2-5g-...
6,OnePlus,"Handset, Power Adapter, USB Cable, SIM Eject Tool",Green Wood,12 GB,256 GB,Primary Camera,-,16.33 cm (6.43 inch),-,"₹34,999",https://www.flipkart.com/oneplus-nord-2-5g-gre...
7,OnePlus,"Handset,SIM Tray Ejector, Adapter, Phone Case,...",Black Dusk,8 GB,128 GB,-,-,16.74 cm (6.59 inch),-,"₹24,999",https://www.flipkart.com/oneplus-nord-ce-2-lit...
8,OnePlus,"Handset,Warp Charge 65 power adapter, Warp Typ...",Gray Sierra,8 GB,128 GB,Primary Camera,-,16.33 cm (6.43 inch),-,"₹29,999",https://www.flipkart.com/oneplus-nord-2-5g-gra...
9,OnePlus,"Phone,SIM Tray Ejector, Adapter, Phone Case, U...",Black Dusk,6 GB,128 GB,Primary & Secondary Camera,-,16.74 cm (6.59 inch),-,"₹19,900",https://www.flipkart.com/oneplus-nord-ce-2-lit...


In [43]:
driver.close

<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="36cfe83be26c656965fd6cb0db98085a")>>

# Ques 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [44]:
driver = webdriver.Chrome(r"C:\Users\Yashika\OneDrive\Desktop\chromedriver.exe")

In [45]:
# opening google maps
url = "https://www.google.co.in/maps"
driver.get(url)
time.sleep(2)


In [47]:
#Sending keyword for seach bar and search button
city = input('Enter City name that has to be searched : ')
search_bar = driver.find_element(By.ID,"searchboxinput")                       
search_bar.clear()                                                             
time.sleep(2)
search_bar.send_keys(city)                                                     
search_btn = driver.find_element(By.ID,"searchbox-searchbutton")              
search_btn.click()                                                             
time.sleep(3)

try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)
    lat_lng = re.findall(r'@(.*)data',url_string)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(lat, lng))

except Exception as e:
        print("Error: ", str(e))

Enter City name that has to be searched : Delhi
URL Extracted:  https://www.google.co.in/maps/place/Delhi/@28.6437954,76.8130345,10z/data=!3m1!4b1!4m6!3m5!1s0x390cfd5b347eb62d:0x37205b715389640!8m2!3d28.7040592!4d77.1024902!16zL20vMDlmMDc
Latitude = 28.6437954, Longitude = 76.8130345


# Ques - 6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in

In [50]:
driver = webdriver.Chrome(r"C:\Users\Yashika\OneDrive\Desktop\chromedriver.exe")

#to get url
driver.get('https://trak.in/')

#to get xpath for funding deals
fund_button = driver.find_element(By.XPATH,'//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(fund_button)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//li[@id="menu-item-51510"]/a"}
  (Session info: chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x00D26643]
	(No symbol) [0x00CBBE21]
	(No symbol) [0x00BBDA9D]
	(No symbol) [0x00BF1342]
	(No symbol) [0x00BF147B]
	(No symbol) [0x00C28DC2]
	(No symbol) [0x00C0FDC4]
	(No symbol) [0x00C26B09]
	(No symbol) [0x00C0FB76]
	(No symbol) [0x00BE49C1]
	(No symbol) [0x00BE5E5D]
	GetHandleVerifier [0x00F9A142+2497106]
	GetHandleVerifier [0x00FC85D3+2686691]
	GetHandleVerifier [0x00FCBB9C+2700460]
	GetHandleVerifier [0x00DD3B10+635936]
	(No symbol) [0x00CC4A1F]
	(No symbol) [0x00CCA418]
	(No symbol) [0x00CCA505]
	(No symbol) [0x00CD508B]
	BaseThreadInitThunk [0x758D7D69+25]
	RtlInitializeExceptionChain [0x77C0BB9B+107]
	RtlClearBits [0x77C0BB1F+191]


In [51]:
# Empty lists
fund_dict = {}
fund_dict['Date'] = []
fund_dict['Startup Name'] = []
fund_dict['Industry/Vertical'] = []
fund_dict['Sub-Vertical'] = []
fund_dict['Location'] = []
fund_dict['Investor'] = []
fund_dict['Investment Type'] = []
fund_dict['Amount(in USD)'] = []


for i in range(54,57):
    # Date
    dt = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in dt:
        fund_dict['Date'].append(d.text)

    # Startup Name
    sn = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in sn:
        fund_dict['Startup Name'].append(n.text)
    
    # Industry/Vertical
    ind = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in ind:
        fund_dict['Industry/Vertical'].append(n.text)
    
    # Sub-Vertical
    sv = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in sv:
        fund_dict['Sub-Vertical'].append(s.text)

    # Location
    loc = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in loc:
        fund_dict['Location'].append(l.text)
    
      # Investor
    inv = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in inv:
        fund_dict['Investor'].append(n.text)
    
    # Investment Type
    invt = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in invt:
        fund_dict['Investment Type'].append(n.text)
    
    # Amount
    amt = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amt:
        fund_dict['Amount(in USD)'].append(a.text)
    
fund_df = pd.DataFrame(fund_dict)
fund_df

,Date,Startup Name,Industry/Vertical,Sub-Vertical,Location,Investor,Investment Type,Amount(in USD)


# Ques - 7. Write a program to scrap all the available details of best gaming laptops from digit.in

In [58]:
driver = webdriver.Chrome(r"C:\Users\Yashika\OneDrive\Desktop\chromedriver.exe")

In [59]:
# opening the url digit.in
url = "https://www.digit.in/"
driver.get(url)
time.sleep(2)

In [60]:
#searching for best laptop
best_gam_laptops = driver.find_element(By.XPATH,"//div[@class='listing_container']//ul//li[9]").click()
time.sleep(4)


In [61]:
#Creating empty lists
Laptop_Name = []
Operating_sys = []
Display = []
Processor = []
Memory = []
Weight = []
Dimensions = []
Graph_proc = []
Price = []

In [62]:
# Scraping the data of laptop names
laptop_name= driver.find_elements(By.XPATH,"//div[@class='right-container']/div/a/h3")
for name in laptop_name:
    Laptop_Name.append(name.text)
    
    
#Scraping the data of operating system
try:
    op_sys = driver.find_elements(By.XPATH,"//div[@class='product-detail']/div/ul/li[1]/div/div")
    for os in op_sys:
        Operating_sys.append(os.text)
except NoSuchElementException:
    pass


#Scraping data of display of the laptop
try:
    display= driver.find_elements(By.XPATH,"//div[@class='product-detail']/div/ul/li[2]/div/div")
    for disp in display:
        Display.append(disp.text)
except NoSuchElementException:
    pass


#Scraping data of Processor
try:
    processor = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[5]/td[3]")
    for pro in processor:
        Processor.append(pro.text)
except NoSuchElementException:
    pass

#Scraping data of memory
try:
    memory = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[3]")
    for memo in memory:
        Memory.append(memo.text)
except NoSuchElementException:
    pass

#Scraping data of weight
try:
    weight = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[7]/td[3]")
    for wgt in weight:
        Weight.append(wgt.text)
except NoSuchElementException:
    pass


#Scraping data of dimensions
try:
    dimension = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[8]/td[3]")
    for dim in dimension:
        Dimensions.append(dim.text)
except NoSuchElementException:
    pass


#Scraping data of Graph processor
try:
    graph = driver.find_elements(By.XPATH,'//div[@class="Spcs-details"]/table/tbody/tr[7]/td[3]')
    for gra in graph:
        Graph_proc.append(gra.text)
except NoSuchElementException:
    pass


#Scraping data of price
try:
    price = driver.find_elements(By.XPATH,"//td[@class='smprice']")
    for pri in price:
        Price.append(pri.text.replace('₹','Rs '))
except NoSuchElementException:
    pass

In [63]:
print(len(Laptop_Name),len(Operating_sys),len(Display),len(Processor),len(Memory),len(Weight),len(Dimensions),len(Graph_proc),len(Price))


0 10 10 10 10 10 9 10 10


In [64]:
df=pd.DataFrame(data,columns=['Laptop_Name','Operating_System','Display','Processor','Memory','Weight','Dimension','Graph_Proc','Price'])


In [66]:
df

,Laptop_Name,Operating_System,Display,Processor,Memory,Weight,Dimension,Graph_Proc,Price


# 8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [67]:
driver = webdriver.Chrome(r"C:\Users\Yashika\OneDrive\Desktop\chromedriver.exe")

In [68]:
#Getting the specified url
url = "https://www.forbes.com/?sh=69e6b8c92254"
driver.get(url)
time.sleep(3)

In [69]:
#clicking on the explore button
button = driver.find_element(By.XPATH,'//div[@class="_69hVhdY4"]')
button.click()
time.sleep(1)

In [70]:
#select billionaire  
bill = driver.find_element(By.XPATH,'//li[@class="TjJgrPSg cD45ib6e primary"]')
bill.click()
time.sleep(1)

In [72]:
#select world billionaire  
world_bill= driver.find_element(By.XPATH,'//li[@class="TjJgrPSg _2bNo56RE secondary"]')
world_bill.click()
time.sleep(1)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//li[@class="TjJgrPSg _2bNo56RE secondary"]"}
  (Session info: chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x00D26643]
	(No symbol) [0x00CBBE21]
	(No symbol) [0x00BBDA9D]
	(No symbol) [0x00BF1342]
	(No symbol) [0x00BF147B]
	(No symbol) [0x00C28DC2]
	(No symbol) [0x00C0FDC4]
	(No symbol) [0x00C26B09]
	(No symbol) [0x00C0FB76]
	(No symbol) [0x00BE49C1]
	(No symbol) [0x00BE5E5D]
	GetHandleVerifier [0x00F9A142+2497106]
	GetHandleVerifier [0x00FC85D3+2686691]
	GetHandleVerifier [0x00FCBB9C+2700460]
	GetHandleVerifier [0x00DD3B10+635936]
	(No symbol) [0x00CC4A1F]
	(No symbol) [0x00CCA418]
	(No symbol) [0x00CCA505]
	(No symbol) [0x00CD508B]
	BaseThreadInitThunk [0x758D7D69+25]
	RtlInitializeExceptionChain [0x77C0BB9B+107]
	RtlClearBits [0x77C0BB1F+191]


In [73]:
#Scraping required data from the web page
#Creatng empty lists
Rank = []
Person_Name = []
total_net_worth = []
Age = []
citizenship = []
Source = []
industry = []

In [74]:
while(True):
    #Scraping the data of rank of the billionaires
    rank_tags= driver.find_elements(By.XPATH,'//div[@class="rank"]')
    for rank in rank_tags:
        Rank.append(rank.text)
    time.sleep(1)
    
    
    #Scraping the data of names of the billionaires
    name_tags= driver.find_elements(By.XPATH,'//div[@class="personName"]/div')
    for name in name_tags:
        Person_Name.append(name.text)
    time.sleep(1)
    
    
    #Scraping data of age of the billionaires
    age_tags= driver.find_elements(By.XPATH,'//div[@class="age"]/div')
    for age in age_tags:
        Age.append(age.text)   
    time.sleep(1)
    
    
    #Scraping data of citizenship of the billionaires
    cit_tags= driver.find_elements(By.XPATH,'//div[@class="countryOfCitizenship"]')
    for cit in cit_tags:
        citizenship.append(cit.text)
    time.sleep(1)
    
    
    #Scraping data of source of income of the billionaires
    sour_tags= driver.find_elements(By.XPATH,'//div[@class="source"]')
    for sour in sour_tags:
        Source.append(sour.text)    
    time.sleep(1)
    
    
    #Scraping data of Industry of the billionaires
    ind_tags= driver.find_elements(By.XPATH,'//div[@class="category"]//div')
    for ind in ind_tags:
        industry.append(ind.text)
        
    
    #scraping data of net_worth of billionaires
    net_tags= driver.find_elements(By.XPATH,'//div[@class="netWorth"]//div[1]')
    for net in net_tags:
        total_net_worth.append(net.text)
    time.sleep(1)
    
    
    #Clicking on next button
    try:
        next_button = driver.find_element(By.XPATH,"//button[@class='pagination-btn pagination-btn--next ']")
        next_button.click()
    except:
        break
        
#Scraping data of net worth
Net_Worth = []
for i in range(0,len(total_net_worth),2):
    Net_Worth.append(total_net_worth[i])

In [75]:
#Framing data
Billionaires=pd.DataFrame({})
Billionaires['Rank'] = Rank
Billionaires['Names'] = Person_Name
Billionaires['Net Worth'] = Net_Worth
Billionaires['Age'] = Age
Billionaires['Citizenship'] = citizenship
Billionaires['Source'] = Source
Billionaires['Industry'] = industry
Billionaires

,Rank,Names,Net Worth,Age,Citizenship,Source,Industry


# 9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [84]:
driver = webdriver.Chrome(r"C:\Users\Yashika\OneDrive\Desktop\chromedriver.exe")

time.sleep(2)
# to get url
url = "https://www.youtube.com/"
driver.get(url)
time.sleep(2)

In [85]:
# input the keywords that user wants to search
user_input = input('Enter the video you want to search : ')

#finding element for search bar
search_bar = driver.find_element(By.XPATH,'//input[@id="search"]')
search_bar.send_keys(user_input)
time.sleep(1)

Enter the video you want to search : RRR


In [86]:
#clicking on search button
search_btn = driver.find_element(By.ID,"search-icon-legacy")  
search_btn.click()
time.sleep(1)

In [87]:
#clicking on first video
link_click = driver.find_element(By.XPATH,"//yt-formatted-string[@class ='style-scope ytd-video-renderer']")
link_click.click()

# 10000 time we scroll down to generate more Comments
for _ in range(10000):
    driver.execute_script("window.scrollBy(0,10000)")

In [88]:
# empty lists
comments = []
comment_time = []
Time = []
Likes = []
No_of_Likes = []

# to scrape comments
cm_tags = driver.find_elements(By.ID,"content-text")
for cm in cm_tags:
    if cm.text is None:
        comments.append("--")
    else:
        comments.append(cm.text)
time.sleep(5)
comments[:500]

['Outstanding',
 'Subirlas en español por favor gracias',
 '',
 'Great movie',
 'Superb movie',
 'Dosti aayese chij hai acche se ho jaye to zindagi ban jati hai or galat se ho jaye to zindagi jahnm ban jata hai very nice movie ️',
 'Sooo  great movie  very ...... Nice  \nI never.... Find a nice movie',
 'Nice',
 'Supar yarr',
 'En español x favor',
 'Mehnat Kam aayegi ek  din',
 'Why tf is there a rewind after every 1 secs?',
 '🇮🇳',
 'Drama',
 'Edit atcha Nehi he yar',
 'Ishu dhruw',
 'H7',
 'Kre jà']

In [89]:
# to scrape time when comment was posted
tm_tags = driver.find_elements(By.XPATH,"//a[contains(text(),'ago')]")
for tm in tm_tags:
    if tm.text is None:
        Time.append("--")
    else:
        Time.append(tm.text)

for i in range(0,len(Time),2):
    comment_time.append(Time[i])
time.sleep(5)
comment_time[:500]

# to scrape comment likes
like_tags = driver.find_elements(By.XPATH,"//span[@class='style-scope ytd-comment-action-buttons-renderer']")
for like in like_tags:
    if like.text is None:
        Likes.append("--")
    else:
        Likes.append(like.text)
    
for i in range(1,len(Likes),2):
    No_of_Likes.append(Likes[i])
Likes[:500] 

['',
 '2',
 '',
 '2',
 '',
 '1',
 '',
 '1',
 '',
 '1',
 '',
 '1',
 '',
 '',
 '',
 '1',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [90]:
#Creating dataframe
Youtube=pd.DataFrame({})
Youtube['Comments'] = comments[:500]
Youtube['Comment_time'] = comment_time[:500]
Youtube['Comment upvotes'] = No_of_Likes[:500]

#Printing dataframe
Youtube

ValueError: Length of values (9) does not match length of index (18)

# 10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description

In [91]:
driver = webdriver.Chrome(r"C:\Users\Yashika\OneDrive\Desktop\chromedriver.exe")
time.sleep(3)

# to get url
url = "https://www.hostelworld.com/"
driver.get(url)

In [92]:
#to scrape location search bar
search_loc = driver.find_element(By.XPATH,'//input[@class="search-input"]')
search_loc.send_keys("London")
time.sleep(2)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//input[@class="search-input"]"}
  (Session info: chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x00D26643]
	(No symbol) [0x00CBBE21]
	(No symbol) [0x00BBDA9D]
	(No symbol) [0x00BF1342]
	(No symbol) [0x00BF147B]
	(No symbol) [0x00C28DC2]
	(No symbol) [0x00C0FDC4]
	(No symbol) [0x00C26B09]
	(No symbol) [0x00C0FB76]
	(No symbol) [0x00BE49C1]
	(No symbol) [0x00BE5E5D]
	GetHandleVerifier [0x00F9A142+2497106]
	GetHandleVerifier [0x00FC85D3+2686691]
	GetHandleVerifier [0x00FCBB9C+2700460]
	GetHandleVerifier [0x00DD3B10+635936]
	(No symbol) [0x00CC4A1F]
	(No symbol) [0x00CCA418]
	(No symbol) [0x00CCA505]
	(No symbol) [0x00CD508B]
	BaseThreadInitThunk [0x758D7D69+25]
	RtlInitializeExceptionChain [0x77C0BB9B+107]
	RtlClearBits [0x77C0BB1F+191]
